In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import pytz

if not mt5.initialize():
    print("MT5 initialization failed:", mt5.last_error())
else:
    print("MT5 initialized successfully.")


rates = mt5.copy_rates_from_pos(
   "XAUUSD",
   mt5.TIMEFRAME_M5,
   0,
   99000
   )


mt5.shutdown()

if rates is None or len(rates) == 0:
    print("No data received. Check symbol name, broker connection, or market availability.")
else:
    print("Data received:", len(rates), "bars")

    df = pd.DataFrame(rates, columns=[
        'time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume'
    ])

    print(df)

MT5 initialized successfully.
Data received: 99000 bars
             time     open     high      low    close  tick_volume  spread  \
0      1704720600  2022.60  2023.32  2021.11  2021.24         2085      16   
1      1704720900  2021.34  2021.40  2020.30  2020.68         1751      16   
2      1704721200  2020.68  2020.96  2019.58  2020.40         1791      16   
3      1704721500  2020.43  2020.70  2019.51  2020.29         1658      16   
4      1704721800  2020.29  2021.36  2020.12  2020.83         2104      16   
...           ...      ...      ...      ...      ...          ...     ...   
98995  1748631600  3288.59  3290.95  3288.48  3290.03          372      17   
98996  1748631900  3289.97  3290.78  3288.02  3288.32          360      17   
98997  1748632200  3288.31  3290.29  3287.72  3289.12          357      17   
98998  1748632500  3289.01  3290.47  3288.78  3289.65          341      17   
98999  1748632800  3289.67  3290.72  3289.30  3289.76          297      17   

       

In [2]:
df['time'] = pd.to_datetime(df['time'], unit='s')

In [4]:
df['time_mt'] = df['time'].dt.tz_localize('Etc/GMT-3')
df['time_ny'] = df['time_mt'].dt.tz_convert('America/New_York')

In [6]:
cols = ['time_mt', 'time_ny'] + [col for col in df.columns if col not in ['time_mt', 'time_ny']]
df = df[cols]

df['time_mt'] = df['time_mt'].dt.tz_localize(None)
df['time_ny'] = df['time_ny'].dt.tz_localize(None)

In [13]:
columns_to_drop = ['time', 'spread', 'real_volume']
df = df.drop(columns=columns_to_drop)

                  time_mt             time_ny     open     high      low  \
0     2024-01-08 13:30:00 2024-01-08 05:30:00  2022.60  2023.32  2021.11   
1     2024-01-08 13:35:00 2024-01-08 05:35:00  2021.34  2021.40  2020.30   
2     2024-01-08 13:40:00 2024-01-08 05:40:00  2020.68  2020.96  2019.58   
3     2024-01-08 13:45:00 2024-01-08 05:45:00  2020.43  2020.70  2019.51   
4     2024-01-08 13:50:00 2024-01-08 05:50:00  2020.29  2021.36  2020.12   
...                   ...                 ...      ...      ...      ...   
98995 2025-05-30 19:00:00 2025-05-30 12:00:00  3288.59  3290.95  3288.48   
98996 2025-05-30 19:05:00 2025-05-30 12:05:00  3289.97  3290.78  3288.02   
98997 2025-05-30 19:10:00 2025-05-30 12:10:00  3288.31  3290.29  3287.72   
98998 2025-05-30 19:15:00 2025-05-30 12:15:00  3289.01  3290.47  3288.78   
98999 2025-05-30 19:20:00 2025-05-30 12:20:00  3289.67  3290.72  3289.30   

         close  tick_volume  
0      2021.24         2085  
1      2020.68         1751

In [16]:
print(df)

                  time_mt             time_ny     open     high      low  \
0     2024-01-08 13:30:00 2024-01-08 05:30:00  2022.60  2023.32  2021.11   
1     2024-01-08 13:35:00 2024-01-08 05:35:00  2021.34  2021.40  2020.30   
2     2024-01-08 13:40:00 2024-01-08 05:40:00  2020.68  2020.96  2019.58   
3     2024-01-08 13:45:00 2024-01-08 05:45:00  2020.43  2020.70  2019.51   
4     2024-01-08 13:50:00 2024-01-08 05:50:00  2020.29  2021.36  2020.12   
...                   ...                 ...      ...      ...      ...   
98995 2025-05-30 19:00:00 2025-05-30 12:00:00  3288.59  3290.95  3288.48   
98996 2025-05-30 19:05:00 2025-05-30 12:05:00  3289.97  3290.78  3288.02   
98997 2025-05-30 19:10:00 2025-05-30 12:10:00  3288.31  3290.29  3287.72   
98998 2025-05-30 19:15:00 2025-05-30 12:15:00  3289.01  3290.47  3288.78   
98999 2025-05-30 19:20:00 2025-05-30 12:20:00  3289.67  3290.72  3289.30   

         close  tick_volume  
0      2021.24         2085  
1      2020.68         1751

In [18]:
df.to_csv('gold_5min_raw_ny.csv', index=False)